# Week 3 Assignment


## Part 1
#### In this part, I will extracting Toronto Neighborhoods information from wiki page and store the information into a dataframe

In [3]:
import requests
import pandas as pd
# !conda install -c anaconda beautifulsoup4 --yes  # uncomment this message if beautiful is not installed
from bs4 import BeautifulSoup

Requesting the information from the website and read it's contents using Beautiful Soup library

In [4]:
webpage = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
contents = BeautifulSoup(webpage.text, 'html.parser')
#contents

By looking at the contents, we found that the information we want are stored under the table attribute, so we find the table using the find method

In [6]:
table = contents.find('table', attrs={'class':'wikitable sortable'})
#table

Looking at the table, we find every entry is stored between the tr tag, and each element in the entry are stored under td tag, so we need to loop through the table fo find all the information. But before that, we need to create an empty dataframe first used to store the information

In [7]:
column_names = ['Postalcode','Borough','Neighborhood']
df = pd.DataFrame(columns = column_names)
df

,Postalcode,Borough,Neighborhood


In the following loop, we loop through each entry and each element in the entry, notice that the second element in each entry, which is the 'Borough' column, can't be 'Not assigned', so we filter that out, also, if the 'Borough' is assigned but the 'Neighborhood' is 'Not assigned' we assign the Borough value to the Neighborhood value

In [8]:
for tr_cell in table.find_all('tr'):
    row = []
    for td_cell in tr_cell.find_all('td'):
        row.append(td_cell.text.strip())
    if len(row)==3 and row[1] != 'Not assigned':
        if row[2] == 'Not assigned':
            row[2] = row[1]
        df.loc[len(df)] = row

In [9]:
df.head(11)

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,Rouge
8,M1B,Scarborough,Malvern
9,M3B,North York,Don Mills North


Next, we create an temp df which we store the information where we group the dataframe by post code and combine the neighborhood in that post code

In [17]:
tmp_df = df.groupby(['Postalcode'])['Neighborhood'].apply(lambda x: "%s" % ', '.join(x)).to_frame()
tmp_df.head()

,Neighborhood
Postalcode,
M1B,"Rouge, Malvern"
M1C,"Highland Creek, Rouge Hill, Port Union"
M1E,"Guildwood, Morningside, West Hill"
M1G,Woburn
M1H,Cedarbrae


In [18]:
tmp_df.reset_index(inplace=True)
tmp_df.head()

,Postalcode,Neighborhood
0,M1B,"Rouge, Malvern"
1,M1C,"Highland Creek, Rouge Hill, Port Union"
2,M1E,"Guildwood, Morningside, West Hill"
3,M1G,Woburn
4,M1H,Cedarbrae


Merge the old dataframe with the new one with the postalcode column, drop the Neighborhood column in the old dataframe and rename the new colomn and we are done!

In [19]:
df_merge = pd.merge(df, tmp_df, on='Postalcode')
df_merge.head()

,Postalcode,Borough,Neighborhood_x,Neighborhood_y
0,M3A,North York,Parkwoods,Parkwoods
1,M4A,North York,Victoria Village,Victoria Village
2,M5A,Downtown Toronto,Harbourfront,Harbourfront
3,M6A,North York,Lawrence Heights,"Lawrence Heights, Lawrence Manor"
4,M6A,North York,Lawrence Manor,"Lawrence Heights, Lawrence Manor"


In [20]:
df_merge.drop(['Neighborhood_x'], axis=1, inplace=True)
df_merge.drop_duplicates(inplace=True)
df_merge.rename(columns={'Neighborhood_y':'Neighborhood'},inplace=True)
df_merge

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
5,M7A,Downtown Toronto,Queen's Park
6,M9A,Queen's Park,Queen's Park
7,M1B,Scarborough,"Rouge, Malvern"
9,M3B,North York,Don Mills North
10,M4B,East York,"Woodbine Gardens, Parkview Hill"
12,M5B,Downtown Toronto,"Ryerson, Garden District"
